### Name :  5/31 作業 2
### Function :  2. 將新聞資料使用NER跑看看

#### 2. 將新聞資料使用NER跑看看，挑出蘋果日報的資料跑NER

In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import gc
import math

##  function Extract_NER():   Extract NER from receiving text list

In [3]:
def Extract_NER(ls_textRow):
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
    
    # 載入 自訂詞典
    User_Dict = {}
    with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
        for udic in UDicts:
            udWord = udic.strip().split(" ")
            if len(udWord) > 1:
                User_Dict[udWord[0]] = udWord[1]
            else:
                User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
    dictionary = construct_dictionary(User_Dict)
 
    ws = WS("./data")
    pos = POS("./data")
    ner = NER("./data")
    
    #  ls_segRow = ws(ls_textRow) ------------   without User Dictionary
    ls_segRow = ws(ls_textRow, 
                   sentence_segmentation=True,
                   segment_delimiter_set = {",", "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    #print("ls_segRow = ", ls_segRow)
    #print("-----------------------------")
    
    del ws
    del WS
    gc.collect()
    ls_posRow = pos(ls_segRow)
    del pos
    del POS
    gc.collect()
    ls_nerRow = ner(ls_segRow,ls_posRow)
    del ner
    del NER
    gc.collect()
    #print("ls_nerRow = ", ls_nerRow)
    #print("-----------------------------")
    return ls_nerRow

## 1. Read data from csv or xlsx file

In [4]:
#讀檔

df_file = pd.read_csv(r"KCC Data/all-news(11554).csv")
print(df_file.columns)
print("-----------------------------------------------------------")
df_file.head(3)

Index(['序號', '日期', '報別', '內容'], dtype='object')
-----------------------------------------------------------


,序號,日期,報別,內容
0,1,2016/4/18,臺灣時報,馬英九總統昨天上午視察衛武營藝術文化中心，文化部長洪孟啟、工程會副主委顏久榮及高雄市副市長吳...
1,2,2016/4/18,臺灣時報,日本九州熊本縣接連遭強震襲擊，台灣分別捐贈熊本縣政府和日本政府款項，高雄市長陳菊、台南市 ...
2,3,2016/4/18,臺灣時報,壽山動物園為增進動物飼養福祉及保育工作，特設立「高雄市壽山動物園認養辦法」設立動物認養專戶...


In [6]:
apple = df_file[df_file.報別 == "蘋果日報"]
apple

,序號,日期,報別,內容
10485,10486,2010/12/26,蘋果日報,高雄市議會正副議長選舉，昨經過近3小時投票，國民黨籍許崑源以3票氣走民進黨的康裕成，拿下議長...
10486,10487,2011/1/3,蘋果日報,五都議長選舉結果日前出爐，個人財產也首度曝光。根據縣市議員最近一次、前年年底向監察院申報的財...
10487,10488,2011/1/1,蘋果日報,高雄地檢署偵辦五都三合一選舉賄選案，昨對7名涉賄選的市議員、25名里長當選人提起當選無效之訴...
10488,10489,2011/1/11,蘋果日報,高雄縣市合併已兩周，高雄市行政系統卻還很混亂，就業服務在高市出現中央與地方兩制，讓民眾霧煞煞...
10489,10490,2011/1/12,蘋果日報,高雄縣市合併後老人福利也受影響，約有一千多名中低收入的生活津貼縮水，從月領六千元減為三千元；...
...,...,...,...,...
10968,10969,2014/10/5,蘋果日報,貝里斯籍油輪「盛昌號」擱淺旗津海岸2個多月，竟成戲水客的海上樂園。《蘋果》近日直擊遊客游上傾...
10969,10970,2014/10/10,蘋果日報,高雄市長選情 馬英九總統前天在國民黨中常會為拉抬楊秋興選情，批高雄市長陳菊施政不用心，並引用...
10970,10971,2014/10/11,蘋果日報,為拉抬國民黨高雄巿長參選人楊秋興選情，馬英九總統日前批陳菊巿政不用心，巿民平均餘命短；高雄綠...
10971,10972,2014/10/20,蘋果日報,偷吃泡麵，竟要賠上千元！收容法院轉介青少年的高雄市慈暉關懷學園，今年八月發現兩名學員偷竊泡麵...


In [10]:
apple.reset_index(inplace=True)
apple

,level_0,index,序號,日期,報別,內容
0,0,10485,10486,2010/12/26,蘋果日報,高雄市議會正副議長選舉，昨經過近3小時投票，國民黨籍許崑源以3票氣走民進黨的康裕成，拿下議長...
1,1,10486,10487,2011/1/3,蘋果日報,五都議長選舉結果日前出爐，個人財產也首度曝光。根據縣市議員最近一次、前年年底向監察院申報的財...
2,2,10487,10488,2011/1/1,蘋果日報,高雄地檢署偵辦五都三合一選舉賄選案，昨對7名涉賄選的市議員、25名里長當選人提起當選無效之訴...
3,3,10488,10489,2011/1/11,蘋果日報,高雄縣市合併已兩周，高雄市行政系統卻還很混亂，就業服務在高市出現中央與地方兩制，讓民眾霧煞煞...
4,4,10489,10490,2011/1/12,蘋果日報,高雄縣市合併後老人福利也受影響，約有一千多名中低收入的生活津貼縮水，從月領六千元減為三千元；...
...,...,...,...,...,...,...
483,483,10968,10969,2014/10/5,蘋果日報,貝里斯籍油輪「盛昌號」擱淺旗津海岸2個多月，竟成戲水客的海上樂園。《蘋果》近日直擊遊客游上傾...
484,484,10969,10970,2014/10/10,蘋果日報,高雄市長選情 馬英九總統前天在國民黨中常會為拉抬楊秋興選情，批高雄市長陳菊施政不用心，並引用...
485,485,10970,10971,2014/10/11,蘋果日報,為拉抬國民黨高雄巿長參選人楊秋興選情，馬英九總統日前批陳菊巿政不用心，巿民平均餘命短；高雄綠...
486,486,10971,10972,2014/10/20,蘋果日報,偷吃泡麵，竟要賠上千元！收容法院轉介青少年的高雄市慈暉關懷學園，今年八月發現兩名學員偷竊泡麵...


## 2.  Main function:
### 2.1  Pass text data to function "Extract_NER" by row
### 2.2  Integrate every 10 NER return rows into one data frame  
### 2.3  Write frame into small csv after droping duplicate, locating class, filtering short NER, sorting by class.. 

In [13]:
#逐筆傳遞欄位 "text"
apple_text = apple["內容"]
print("內容 欄位筆數 = ",len(apple_text))
print("------------------------------------------")
ls_class = list()
ls_ner = list()
fileCunt = 0
rowCunt = 0
for i in range(len(apple_text)):
    rowCunt += 1
    ls_dfRow = [apple_text[i]]
    # print("ls_dfRow = ", ls_dfRow)
    print("Row No = ",str(i))
    entity_list = Extract_NER(ls_dfRow)
    for j in range(len(entity_list[0])):
        seg = entity_list[0].pop()
        # print("seg = ",seg)
        ls_class.append(seg[2])
        ls_ner.append(seg[3])
    #print("===========================================")
    #print("ls_class = ",ls_class)
    #print("ls_ner = ",ls_ner)
    # if i == 5 or rowCunt == 2:
    if i == (len(apple_text)-1) or rowCunt == 10:
        fileCunt += 1
        extract_df = pd.DataFrame({'Class':ls_class, 'NER':ls_ner})
        deDup_df = extract_df.drop_duplicates().copy()
        del extract_df #去除重複
        cate_df = deDup_df.loc[deDup_df['Class'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','NORP'])].copy()
        del deDup_df
        filt_df = cate_df[cate_df['NER'].map(len) >= 2].copy()
        del cate_df
        NER_df = filt_df.sort_values(by=['Class']).copy()
        del filt_df
        #print("__________________________________________________________")  
        #print(NER_df)
        print("fileCunt No = ",str(fileCunt))
        NER_df.to_csv(r"News Data\News NER Out\\NewsNER870_" + str(fileCunt)+ ".csv")
        #NER_df.to_csv(r"ner out/NerTest_" + str(fileCunt)+ ".csv")
        rowCunt = 0
print("=====================================================================")
print("Totle NER Files = ",fileCunt)


內容 欄位筆數 =  488
------------------------------------------
Row No =  0


C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  1
Row No =  2
Row No =  3
Row No =  4
Row No =  5
Row No =  6
Row No =  7
Row No =  8
Row No =  9
fileCunt No =  1
Row No =  10


C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  11
Row No =  12
Row No =  13
Row No =  14
Row No =  15
Row No =  16
Row No =  17
Row No =  18
Row No =  19
fileCunt No =  2
Row No =  20


...

Row No =  461
Row No =  462
Row No =  463
Row No =  464
Row No =  465
Row No =  466
Row No =  467
Row No =  468
Row No =  469
fileCunt No =  47
Row No =  470


C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  471
Row No =  472
Row No =  473
Row No =  474
Row No =  475
Row No =  476
Row No =  477
Row No =  478
Row No =  479
fileCunt No =  48
Row No =  480


C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\CHING TING\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  481
Row No =  482
Row No =  483
Row No =  484
Row No =  485
Row No =  486
Row No =  487
fileCunt No =  49
Totle NER Files =  49


## 3. Integrate all csv files into single data frame

In [15]:
import glob
# import pandas as pd

path = r'News Data\News NER OUT'
#path = r'ner out'
allFiles = glob.glob("{}/*.csv".format(path))
li = []
for filename in allFiles:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

nerFrame = pd.concat(li, axis=0, ignore_index=True)
print(len(nerFrame))
nerFrame.groupby('Class').count()

58255


,NER
Class,
EVENT,1096
GPE,7997
LAW,1260
LOC,4208
NORP,546
ORG,20901
PERSON,22247


## 4. Drop duplicates, group by class and write into csv file 

In [16]:
ddFrame = nerFrame.drop_duplicates().copy()
print(len(ddFrame))
ddFrame.groupby('Class').count()

2061


,NER
Class,
EVENT,44
GPE,281
LAW,49
LOC,167
NORP,25
ORG,736
PERSON,759


In [18]:
ddFrame.sort_values(by=['Class']).to_csv(r"News Data\News NER OUT\\NewsNER.csv")
#ddFrame.sort_values(by=['Class']).to_csv(r"ner out/NerTest.csv")